# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 11, 12, 10

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is buy
Number of combinations is 60


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [06:05<00:00,  6.09s/it]


# Check local statistics

In [2]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
21,PriceChange_LinearReg,8,6,0,97.28,91.30,86.96,86.96,86.96,83.70,82.07,83.70,82.07,80.43,80.98,79.89,80.98,82.61,81.52,85.33,86.41,84.78,85.87,85.87,86.96,88.59,91.30,88.59,0.99,2.04,1.75,1.69,1.87,2.18,1.88,1.92,2.43,2.34,2.03,2.34,4.22,4.26,4.12,3.91,3.63,3.57,3.38,3.53,3.73,4.08,3.21,3.68,184,85.462917,2.865833,2845.176667,527.313333
27,PriceChange_LinearReg,10,6,0,95.95,90.54,86.94,84.23,84.68,82.88,81.08,82.43,81.98,80.18,80.18,79.73,80.63,83.33,81.98,85.59,86.49,86.04,85.59,85.14,86.94,88.74,91.89,88.74,0.96,1.97,1.63,1.55,1.78,2.07,1.74,1.72,2.39,2.25,2.02,2.24,4.18,4.22,4.04,3.78,3.59,3.55,3.35,3.49,3.64,3.95,3.17,3.59,222,85.079167,2.786250,3347.575000,618.547500
22,PriceChange_LinearReg,8,8,0,96.88,91.15,86.46,86.98,86.98,83.33,80.73,81.77,80.21,78.65,78.65,78.65,80.21,81.25,80.73,84.38,84.90,85.42,85.42,85.94,86.46,88.54,90.62,88.02,0.99,2.03,1.72,1.69,1.86,2.13,1.85,1.87,2.42,2.33,2.02,2.32,4.18,4.24,4.10,3.93,3.61,3.57,3.39,3.56,3.73,4.04,3.22,3.65,192,84.680417,2.852083,2818.640000,547.600000
36,PriceChange_LinearReg,13,6,0,94.64,89.64,87.50,85.00,84.64,82.86,80.71,80.71,80.71,78.21,78.57,78.21,80.00,83.21,82.50,85.36,86.43,86.07,85.71,84.64,86.79,88.57,91.79,88.93,0.96,1.91,1.59,1.54,1.73,1.99,1.62,1.63,2.19,2.02,1.93,2.16,4.02,3.96,3.98,3.68,3.47,3.40,3.18,3.39,3.59,3.90,3.14,3.39,280,84.641667,2.682083,4099.666667,750.983333
33,PriceChange_LinearReg,12,6,0,95.02,89.27,86.97,84.29,83.91,82.38,80.46,80.84,81.23,78.93,79.31,79.31,81.23,83.91,82.38,84.67,86.21,85.44,85.44,84.29,86.21,87.74,91.57,88.12,0.97,1.91,1.57,1.53,1.73,2.02,1.64,1.64,2.19,2.05,1.94,2.17,4.07,4.00,3.99,3.69,3.45,3.45,3.20,3.38,3.57,3.82,3.08,3.40,261,84.547083,2.685833,3796.788750,701.002500
30,PriceChange_LinearReg,11,6,0,95.42,90.42,87.08,84.17,84.17,81.67,80.00,80.83,80.83,78.75,79.17,78.75,80.42,83.33,82.08,85.00,85.83,85.83,85.42,85.00,86.25,87.92,91.67,88.75,0.95,1.93,1.61,1.53,1.74,2.00,1.69,1.68,2.26,2.11,1.99,2.23,4.11,4.13,4.00,3.75,3.53,3.47,3.23,3.42,3.61,3.94,3.14,3.51,240,84.531667,2.731667,3487.600000,655.600000
24,PriceChange_LinearReg,9,6,0,96.53,90.59,86.14,85.15,85.15,82.67,80.20,82.18,81.19,79.70,80.20,79.21,79.70,81.19,80.69,84.65,85.15,84.65,84.16,83.66,86.14,87.62,91.09,88.12,0.99,1.98,1.65,1.60,1.81,2.08,1.79,1.74,2.41,2.29,2.03,2.24,4.18,4.23,4.06,3.80,3.60,3.53,3.31,3.43,3.63,3.95,3.16,3.60,202,84.405417,2.795417,2909.894167,564.674167
28,PriceChange_LinearReg,10,8,0,95.74,90.64,87.23,84.68,84.68,82.13,80.00,81.28,80.00,78.30,77.87,78.72,80.00,82.13,81.28,84.68,85.11,85.96,84.68,84.68,86.81,88.94,91.49,88.51,0.96,1.95,1.62,1.54,1.76,2.02,1.72,1.67,2.34,2.19,2.02,2.23,4.15,4.22,4.03,3.78,3.56,3.52,3.34,3.48,3.63,3.95,3.18,3.56,235,84.397500,2.767500,3383.412500,650.362500
37,PriceChange_LinearReg,13,8,0,94.50,89.35,87.29,84.88,84.19,81.79,80.41,80.07,79.73,76.98,77.66,78.01,80.07,83.16,82.47,85.57,85.91,86.25,85.22,84.19,86.60,88.66

# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [10]}, 
              'LinearReg': {'timeperiod': [6], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278
